# INNER JOIN e LEFT JOIN

A característica mais marcante do SQL é, sem dúvida, a junção, pois é para isso que os bancos de dados relacionais foram projetados. Embora existam vários tipos de junção, nenhum é tão comumente usado quanto o `INNER JOIN` e o `LEFT JOIN`. Abordaremos esses dois operadores nesta seção.

## Configuração

Primeiro, configure. Baixe o arquivo de banco de dados SQLite `company_operations.db` e conecte-se a ele. Também inclua `pandas` para exibir os resultados da nossa consulta SQL como um `DataFrame`.

In [ ]:
import sqlite3
import pandas as pd
import urllib.request

# baixe o banco de dados SQLite e conecte-se a ele
urllib.request.urlretrieve("https://github.com/thomasnield/anaconda_intro_to_sql/blob/main/company_operations.db?raw=true", "company_operations.db")
conn = sqlite3.connect('company_operations.db')

## Chaves primárias e estrangeiras

Vamos dar uma olhada em duas tabelas: `CUSTOMER` e `CUSTOMER_ORDER`. O que elas têm em comum?

In [ ]:
sql = """
SELECT * FROM CUSTOMER
"""

pd.read_sql(sql, conn)


In [ ]:
sql = """
SELECT * FROM CUSTOMER_ORDER
"""

pd.read_sql(sql, conn)


As duas tabelas têm uma coluna `CUSTOMER_ID`, e você provavelmente pode inferir o que ela faz para cada tabela. A tabela `CUSTOMER` tem um `CUSTOMER_ID` exclusivo atribuído a cada registro de cliente. Mas na tabela `CUSTOMER_ORDER`, ele é usado para atribuir um pedido a um determinado `CUSTOMER`, usando esse `CUSTOMER_ID`.

Isso torna a tabela `CUSTOMER` a **tabela pai**, com `CUSTOMER_ID` sendo a **chave primária**. A tabela `CUSTOMER_ORDER` é a **tabela filha**, com `CUSTOMER_ID` sendo a **chave estrangeira**. Você pode pensar nela como se a tabela pai *fornecessário* dados para a tabela filha, por meio da chave primária para a chave estrangeira.

A chave primária não pode ter valores duplicados, e isso faz sentido, já que dois clientes não devem ter o mesmo `CUSTOMER_ID`. No entanto, pode haver várias instâncias de um valor em uma coluna de chave estrangeira, pois um determinado cliente pode ter vários pedidos. Este é um clássico relacionamento um-para-muitos.

Esses relacionamentos são projetados para serem unidos e são um caso de uso fundamental para `INNER JOIN` e `LEFT JOIN`.

## INNER JOIN

O `INNER JOIN` é o tipo mais comum de join em SQL. Ele une duas ou mais tabelas em um ou mais campos. Em nosso exemplo, seria útil tornar nossos registros `CUSTOMER_ORDER` mais descritivos, trazendo as informações `CUSTOMER` junto com cada registro `CUSTOMER_ORDER`. Um `INNER JOIN` pode fazer isso, como mostrado abaixo.

In [ ]:
sql = """
SELECT 
CUSTOMER_ORDER_ID,
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
ADDRESS,
CITY,
STATE,
ZIP,
ORDER_DATE,
PRODUCT_ID,
QUANTITY

FROM CUSTOMER INNER JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID
"""

pd.read_sql(sql, conn)


Acima, extraímos campos de `CUSTOMER` e `CUSTOMER_ORDER` e, como o `CUSTOMER_ID` existe em ambas as tabelas, escolhemos uma usando a sintaxe `CUSTOMER.CUSTOMER_ID` para selecionar a da tabela `CUSTOMER`.

> Para campos ambíguos como `CUSTOMER_ID`, é uma boa regra geral preferir o da tabela pai, caso você troque seu `INNER JOIN` por um `LEFT JOIN` à esquerda, para que ele nunca se torne `NULL`. Aprenderemos sobre isso mais tarde.

O `INNER JOIN` nos permite unir as duas tabelas e definir a similaridade usando a palavra-chave `ON`, que é onde nossa condição de junção se encaixa. Você pode especificar qualquer condição que desejar para um `JOIN`, mas 99,9% das vezes você provavelmente usará um simples sinal de igual `=` para alinhar os registros entre as duas tabelas.

Outra maneira de pensar nisso é que estamos copiando cada registro `CUSTOMER` em cada `CUSTOMER_ORDER` respectivo, compartilhando o respectivo `CUSTOMER_ID`.

> Ocasionalmente, você pode encontrar colegas que simplesmente escrevem `JOIN` em vez de `INNER JOIN`. Esta é uma abreviação ou alias para `INNER JOIN`, mas é uma prática recomendada expressar explicitamente `INNER JOIN` para deixar claro que esse era o tipo de junção pretendido.

Você também deve evitar usar uma convenção antiga de junção interna, selecionando tabelas separadas por vírgulas e usando a condição `WHERE` para sua condição de junção, conforme mostrado abaixo. Esta é uma [sintaxe inflexível e confusa, menos legível](https://stackoverflow.com/questions/1018822/inner-join-on-vs-where-clause), e eu recomendo evitá-la.

In [ ]:
sql = """
SELECT 
CUSTOMER_ORDER_ID,
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
ADDRESS,
CITY,
STATE,
ZIP,
ORDER_DATE,
PRODUCT_ID,
QUANTITY

FROM CUSTOMER, CUSTOMER_ORDER
WHERE CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID
"""

pd.read_sql(sql, conn)


## LEFT JOIN

O que acontece se houver registros `CUSTOMER` que não tenham nenhum registro `CUSTOMER_ORDER`? Eles aparecem em um `INNER JOIN`? Por exemplo, "Alpha Medical" com `CUSTOMER_ID` igual a 1 não possui nenhum pedido. Ele aparece em nossa consulta `INNER JOIN`? Vamos adicionar uma condição `WHERE` para descobrir.

In [ ]:
sql = """
SELECT 
CUSTOMER_ORDER_ID,
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
ADDRESS,
CITY,
STATE,
ZIP,
ORDER_DATE,
PRODUCT_ID,
QUANTITY

FROM CUSTOMER INNER JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

WHERE CUSTOMER.CUSTOMER_ID = 1
"""

pd.read_sql(sql, conn)


Com certeza, obtemos um resultado vazio. Mas veja o que acontece se mudarmos nosso `INNER JOIN` para `LEFT JOIN` (ou `LEFT OUTER JOIN`, que são ambos aliases para a mesma operação).

In [ ]:
sql = """
SELECT 
CUSTOMER_ORDER_ID,
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
ADDRESS,
CITY,
STATE,
ZIP,
ORDER_DATE,
PRODUCT_ID,
QUANTITY

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

WHERE CUSTOMER.CUSTOMER_ID = 1 
"""

pd.read_sql(sql, conn)


Observe como "Alpha Medical" agora aparece com um registro de espaço reservado, mesmo sem nenhum registro `CUSTOMER_ORDER`. Todos os seus campos `CUSTOMER_ORDER` são `NULL` (que o Pandas está exibindo como `None`) porque não havia registros `CUSTOMER_ORDER` para unir e preencher essas informações. Mas o `LEFT JOIN` anexou este registro de espaço reservado para "Alpha Medical".

Em outras palavras, o `LEFT JOIN` inclui todos os registros na tabela "esquerda", mesmo que não haja registros para unir na tabela "direita". Por "esquerda", quero dizer a tabela literalmente especificada à esquerda do operador `LEFT JOIN`. Isso significa que a ordem em que você declara as tabelas em `FROM` importa com um `LEFT JOIN`.

> Há também um operador `RIGHT JOIN` ou `RIGHT OUTER JOIN`, que inverte a direção e inclui todos os registros na tabela `RIGHT`, mesmo que não haja nenhum para unir na tabela `LEFT`. No entanto, ele raramente é usado, pois o que pode ser feito com um `RIGHT JOIN` também pode ser alcançado com um `LEFT JOIN`. Há também um `FULL OUTER JOIN` que inclui todos os registros em ambas as direções, mas também raramente é usado. Aliás, o SQLite não suporta `RIGHT JOIN` ou `FULL OUTER JOIN` por esse motivo.

Como veremos, isso pode ser útil para criar relatórios posteriormente, pois provavelmente queremos incluir clientes que não têm pedidos. Outro caso de uso comum para `LEFT JOIN` é encontrar registros pai que não têm filhos, como registros `CUSTOMER` que não têm nenhum registro `CUSTOMER_ORDER`. Podemos fazer isso qualificando quaisquer campos `CUSTOMER_ORDER` como nulos, que normalmente não são nulos, mas consequentemente se tornam nulos como resultado do `LEFT JOIN`.

In [ ]:
sql = """
SELECT 
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER_ORDER.CUSTOMER_ID = CUSTOMER.CUSTOMER_ID

WHERE CUSTOMER_ORDER.CUSTOMER_ID IS NULL
"""

pd.read_sql(sql, conn)


## Unindo várias tabelas

E se quiséssemos trazer informações de `PRODUCT` para nossos registros `CUSTOMER_ORDER`, além de informações de `CUSTOMER`? Isso é possível executando uma segunda junção. Vamos dar uma olhada na tabela `PRODUCT` e observar que ela usa um `PRODUCT_ID`, que também existe na tabela `CUSTOMER_ORDER` como chave estrangeira.

In [ ]:
sql = """
SELECT * FROM PRODUCT
"""

pd.read_sql(sql, conn)


Vamos incluir `PRODUCT_NAME` e `PRICE` para exibir ao lado de cada `CUSTOMER_ORDER`. Podemos executar uma segunda junção no `PRODUCT_ID` e costurar a tabela de informações `PRODUCT` à nossa malha existente entre `CUSTOMER_ORDER` e `CUSTOMER`.

In [ ]:
sql = """
SELECT 
CUSTOMER_ORDER_ID,
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
ADDRESS,
CITY,
STATE,
ZIP,
ORDER_DATE,
PRODUCT.PRODUCT_ID,
QUANTITY, 
PRICE

FROM CUSTOMER INNER JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

INNER JOIN PRODUCT
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID
"""

pd.read_sql(sql, conn)


Você pode misturar `INNER JOIN` e `LEFT JOIN` em uma consulta, e precisa analisar cuidadosamente esses cenários, pois isso se torna específico para cada caso de uso. Mas, neste cenário, se quiséssemos incluir todos os registros `CUSTOMER`, precisaríamos usar `LEFT JOIN` para ambas as junções, incluindo `PRODUCT`, porque os valores nulos da primeira `LEFT JOIN` serão propagados para a próxima junção. A segunda `LEFT JOIN` tolerará esses valores nulos, mas a `INNER JOIN` não, e simplesmente os omitirá.

In [ ]:
sql = """
SELECT 
CUSTOMER_ORDER_ID,
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
ADDRESS,
CITY,
STATE,
ZIP,
ORDER_DATE,
PRODUCT.PRODUCT_ID,
QUANTITY, 
PRICE

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

LEFT JOIN PRODUCT
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID
"""

pd.read_sql(sql, conn)


## Agregando joins

Se você pensar nas consultas acima que executamos anteriormente como "novas" tabelas produzidas pelas junções, logicamente podemos aplicar `GROUP BY` a elas, bem como funções de agregação como `SUM()`. Se quisermos encontrar a receita total por cliente, vamos adicionar uma expressão `PRICE * QUANTITY` e chamá-la de `REVENUE`.

In [ ]:
sql = """
SELECT
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
PRICE * QUANTITY AS REVENUE

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

LEFT JOIN PRODUCT
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID
"""

pd.read_sql(sql, conn)


Então podemos `SUM()` essa expressão e adicionar `GROUP BY` para agrupar os atributos `CUSTOMER`.

In [ ]:
sql = """
SELECT
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
SUM(PRICE * QUANTITY) AS TOTAL_REVENUE

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

LEFT JOIN PRODUCT
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID

GROUP BY CUSTOMER.CUSTOMER_ID, CUSTOMER_NAME
"""

pd.read_sql(sql, conn)


Por fim, podemos `COALESCE()` em `TOTAL_REVENUE` para transformar quaisquer valores nulos em `0`. E pronto!

In [ ]:
sql = """
SELECT
CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME,
COALESCE(SUM(PRICE * QUANTITY), 0) AS TOTAL_REVENUE

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

LEFT JOIN PRODUCT
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID

GROUP BY CUSTOMER.CUSTOMER_ID, CUSTOMER_NAME
"""

pd.read_sql(sql, conn)


E é isso! Aprendemos os fundamentos das junções SQL. Se você se sentir confortável com essa operação, poderá se considerar um desenvolvedor SQL.

## EXERCÍCIO

Encontre a receita total por produto completando a consulta abaixo, substituindo os pontos de interrogação "?" pelo SQL apropriado.

In [ ]:
sql = """
SELECT
PRODUCT.PRODUCT_ID, 
PRODUCT_NAME,
COALESCE(SUM(PRICE * QUANTITY), 0) AS TOTAL_REVENUE

FROM ?

GROUP BY PRODUCT.PRODUCT_ID, PRODUCT_NAME
"""

pd.read_sql(sql, conn)


### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
sql = """
SELECT
PRODUCT.PRODUCT_ID, 
PRODUCT_NAME,
COALESCE(SUM(PRICE * QUANTITY), 0) AS TOTAL_REVENUE

FROM PRODUCT LEFT JOIN CUSTOMER_ORDER
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID

GROUP BY PRODUCT.PRODUCT_ID, PRODUCT_NAME
"""

pd.read_sql(sql, conn)
